Imports


In [1]:
import gc
import os
import re
import torch
import warnings
from ultralytics import YOLO

Disabling Warnings


In [2]:
warnings.filterwarnings("ignore")

Setting Training Device


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if str(device) == "cuda":
    print(f"Using GPU {torch.cuda.get_device_properties(device)}")
else:
    print("Using CPU")

Using GPU _CudaDeviceProperties(name='AMD Radeon RX 7800 XT', major=11, minor=0, gcnArchName='gfx1100', total_memory=16368MB, multi_processor_count=30, uuid=38336232-6265-3432-3662-306564613532, L2_cache_size=4MB)


Emptying the GPU Cache (if necessary)


In [4]:
def empty_cache() -> None:
    # Cleaning out the device cache
    gc.collect()
    torch.cuda.empty_cache()


def print_free_memory() -> None:
    free, total = torch.cuda.mem_get_info(device)
    print(f"Percent of free memory: {round(free/total *100,2)}")


empty_cache()
print_free_memory()

Percent of free memory: 94.13


Memory Summary


In [5]:
def memory_summary() -> None:
    print(torch.cuda.memory_summary())


memory_summary()

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |

Preparing GPU (if necessary)


In [6]:
# For AMD GPU - 7800xt
device_name = torch.cuda.get_device_name(0)
if "AMD" in device_name or "Radeon" in device_name:
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = "11.0.0"

print(f"GPU {torch.cuda.get_device_properties(device).name} is now setup")

GPU AMD Radeon RX 7800 XT is now setup


#####


Setting Paths for Data, Base Model, and Output Directory


In [ ]:
# Directory Paths
training_dataset_name, validation_dataset_name, data_method = "coco8", "validation_dataset", "clean"
current_directory = os.getcwd()
path_to_base_directory = re.search(rf"(.*?){"Weird-Stuff-In-Traffic"}", current_directory).group(1)
validation_yaml_data_path = f"Weird-Stuff-In-Traffic/Data/yolo/{training_dataset_name}/coco8.yaml"
complete_validation_data_path = path_to_base_directory + validation_yaml_data_path

# Training Configuration used for validation results path
model_training_config = f"{training_dataset_name}_{data_method}"

# Model Paths
pretrained_model_path = f"Weird-Stuff-In-Traffic/Models/Segmentation-Detection/yolo/{training_dataset_name}_{data_method}/experiment"
full_pretrained_model_path = path_to_base_directory + pretrained_model_path + "/weights/best.pt"

# Results Path
validation_results_path = f"Weird-Stuff-In-Traffic/Results/Segmentation-Detection/yolo/"
complete_validation_results_path = path_to_base_directory + validation_results_path + model_training_config

Loading Pretrained Model


In [10]:
model = YOLO(full_pretrained_model_path)

Model Validation


In [12]:
metrics = model.val(save_json=True, project= complete_validation_results_path)

Ultralytics 8.3.103 🚀 Python-3.12.3 torch-2.7.0.dev20250127+rocm6.3 CUDA:0 (AMD Radeon RX 7800 XT, 16368MiB)
YOLO11n summary (fused): 100 layers, 2,616,248 parameters, 0 gradients, 6.5 GFLOPs


val: Scanning /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Data/yolo/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.73it/s]


                   all          4         17      0.565       0.85      0.849      0.632
                person          3         10      0.549        0.6      0.595      0.276
                   dog          1          1      0.543          1      0.995      0.697
                 horse          1          2      0.528          1      0.995      0.674
              elephant          1          2      0.365        0.5      0.519      0.257
              umbrella          1          1      0.564          1      0.995      0.995
          potted plant          1          1       0.84          1      0.995      0.895
Speed: 0.1ms preprocess, 4.4ms inference, 0.0ms loss, 24.1ms postprocess per image
Saving /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Results/Segmentation-Detection/yolo/coco8_clean/fine_tuned_model/val2/predictions.json...
Results saved to /home/tom/Desktop/Programming/Shared/Weird-Stuff-In-Traffic/Results/Segmentation-Detection/yolo/coco8_clean/fine_tuned_m